# Importing the libs

In [62]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importing the datasets

In [63]:
# We won't be using this dataset.
movies = pd.read_csv('data/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('data/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('data/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

# Prepare the training & test sets

In [64]:
training_set = pd.read_csv('data/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('data/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

# Get the number of users and movies

In [65]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_users

943

In [66]:
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))
nb_movies

1682

# Converting & preparing data

In [67]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
      
    # Get the current user's movie ids
    id_movies = data[:, 1][data[:, 0] == id_users]
    
    # Get the current user's rating scores 
    id_ratings = data[:, 2][data[:, 0] == id_users]
    
    ratings = np.zeros(nb_movies)
    # Assign the ratings to movies, which they belong
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

# Convert data to tensors

In [68]:
training_set = torch.FloatTensor(training_set)
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [69]:
test_set = torch.FloatTensor(test_set)
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

# Create the SAE Class

In [70]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [71]:
# Create instances and define parameters
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

# Training

In [72]:
# Define number of epoch
nb_epoch = 200

In [74]:
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  # s -> number of users who rated at least one movie
  # set it float to observe RMSE
  s = 0.
  for id_user in range(nb_users):
    # To add a fake dimension in first position we use Variable squeeze 0
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    # torch.sum(target.data > 0) > 0, Means that user rated at least one movie
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      # We don't compute gradient with respect to target, save computation
      target.require_grad = False
      # Save computation
      output[target == 0] = 0
      # Calculate the loss with output and target
      loss = criterion(output, target)
      # Make sure that we don't divide with 0
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      # Decide the step direction
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      # Decide the step intensity
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.7661)
epoch: 2loss: tensor(1.0966)
epoch: 3loss: tensor(1.0533)
epoch: 4loss: tensor(1.0382)
epoch: 5loss: tensor(1.0311)
epoch: 6loss: tensor(1.0268)
epoch: 7loss: tensor(1.0238)
epoch: 8loss: tensor(1.0219)
epoch: 9loss: tensor(1.0207)
epoch: 10loss: tensor(1.0198)
epoch: 11loss: tensor(1.0187)
epoch: 12loss: tensor(1.0183)
epoch: 13loss: tensor(1.0179)
epoch: 14loss: tensor(1.0174)
epoch: 15loss: tensor(1.0172)
epoch: 16loss: tensor(1.0169)
epoch: 17loss: tensor(1.0166)
epoch: 18loss: tensor(1.0163)
epoch: 19loss: tensor(1.0163)
epoch: 20loss: tensor(1.0163)
epoch: 21loss: tensor(1.0161)
epoch: 22loss: tensor(1.0160)
epoch: 23loss: tensor(1.0159)
epoch: 24loss: tensor(1.0160)
epoch: 25loss: tensor(1.0156)
epoch: 26loss: tensor(1.0156)
epoch: 27loss: tensor(1.0153)
epoch: 28loss: tensor(1.0151)
epoch: 29loss: tensor(1.0129)
epoch: 30loss: tensor(1.0117)
epoch: 31loss: tensor(1.0095)
epoch: 32loss: tensor(1.0093)
epoch: 33loss: tensor(1.0047)
epoch: 34loss: tens